# Test code to implement paralellization

In [1]:
#!/usr/bin/env python
# coding: utf-8
'''
======================================================
Author:  Ömer Özak, 2014 (ozak at smu.edu)
Website: http://omerozak.com
GitHub:  https://github.com/ozak/
======================================================
Program to compute all geographical statistics for regions given a shapefile
'''
from __future__ import division, print_function
import sys, os
#from osgeo import gdal, gdalnumeric, ogr, osr
#from gdalconst import *
#from PIL import Image, ImageDraw
#import tarfile
#import gzip
#import shutil, glob
import pandas as pd
#from pyGDsandbox.dataIO import df2dbf, dbf2df
from rasterstats import zonal_stats
#import pysal as ps
#import shapely
#from shapely.wkt import loads, dumps
#from pysal.contrib import shapely_ext
import numpy as np
from rtree import index
from shapely.ops import unary_union, linemerge
import isounidecode
import geopandas as gp
from geopandas.tools import sjoin, overlay
import pyproj
from pyproj import CRS
#import hmi
#import georasters as gr
import re

# Determine drive where other data is located
try:
    os.listdir("/Volumes/Mirror RAID")
    drv="/Volumes/Backup/MacPro/Mirror RAID/"
except:
    drv="/Users/bizcocho/Desktop/"

# Fix paths of Ethnic, country and coastline Shapefiles
# Although we include the WGS84 versions, we will only use the projected ones, in order to have everything
if sys.platform=='darwin':
    path = os.getenv('HOME') + '/Google Drive/LatexMe/CulturalDistance/'
    pathcyl=path + '/data/GIS/cyl/'
    pathcntry=path + 'data/GIS/GMI/'
    pathcntrycyl=path + 'data/GIS/GMI/cyl/'
    pathout=path + 'data/GIS/data/'
    pathgis=path + 'data/GIS/'
    pathmeasures = {'Suitability' : pathgis + 'Ramankutty/',
                    'Lights' : '/Volumes/My Book/GIS/Lights/tifs/',
                    'Lights2' : '/Volumes/My Book/GIS/Lights/tifs/',
                    'Elevation' : drv + '/Geographical_Index/GIS/GLOBE/',
                    'Elevation2' : drv + '/Geographical_Index/GIS/GLOBE/',
                    'RIX' : drv + '/Geographical_Index/GIS/RIX/',
                    'RIX2' : drv + '/Geographical_Index/GIS/RIX/',
                    'CRU' : drv + '/Geographical_Index/DataOthers/CRU/tif/',
                    'CropsYield' : drv + '/Geographical_Index/DataOthers/FAO/GAEZ/Agroclimatic/yield/rain_fed/',
                    'CSICrops' : drv + '/Geographical_Index/DataOthers/FAO/GAEZ/Agroclimatic/energyyield/rain_fed/',
                    'CSI' : drv + '/Geographical_Index/DataOthers/FAO/GAEZ/Agroclimatic/bestcropcal/CSI/',
                    'CSIWater' : drv + '/Geographical_Index/DataOthers/FAO/GAEZ/Agroclimatic/bestcropcal/CSI_water/',
                    'CSICycle' : drv + '/Geographical_Index/DataOthers/FAO/GAEZ/Agroclimatic/bestcropcal/updated/',
                    'CSICycleWater' : drv + '/Geographical_Index/DataOthers/FAO/GAEZ/Agroclimatic/bestcropcal/updated_water/',
                    'CSIPlow' : drv + '/Geographical_Index/DataOthers/FAO/GAEZ/Agroclimatic/bestcropcal/plow/',
                    'Malaria' : pathgis + 'Malaria/',
                    'HMI' : drv + '/Geographical_Index/GIS/HMI_Diff/',
                    'PopDensAfrica' : '/Volumes/Macintosh HD 2/GIS/UNEP/',
                    'Popdens' : '/Volumes/My Book/GIS/GPW/popdens/',
                    'Population' : '/Volumes/My Book/GIS/GPW/population/',
                    'HYDE' : '/Volumes/Backup/MacPro/Macintosh HD 2/GIS/HYDE/hyde31_final/tif/',
                    'Tsetse' : pathgis + 'Tsetse/',
                    'Ecodiversity' : pathgis + 'Ecological_Zones/',
                    'EcodiversityLGM' : pathgis + 'PaleoVegetation/',
                    'Sea100' : pathcntrycyl,
                    'Coast' : pathcntrycyl,
                    'Inwater' : pathcntrycyl,
                    'PerInwater' : pathcntrycyl,
                    'FluctInwater' : pathcntrycyl,
                    'Landscan' : '/Volumes/Backup/MacPro/Macintosh HD 2/GIS/Landscan/',
                    'GEcon' : '/Volumes/Backup/MacPro/Macintosh HD 2/GIS/GEconRaster/',
                    'HLD' : '/Volumes/My Book/GIS/Harmonized Light Data/light/',
                    }
elif sys.platform.startswith('linux'):
    path = os.getenv('HOME') + '/LatexMe/CulturalDistance/'
    pathcyl=path + '/data/GIS/cyl/'
    pathcntry=path + 'data/GIS/GMI/'
    pathcntrycyl=path + 'data/GIS/GMI/cyl/'
    pathout=path + 'data/GIS/data/'
    pathgis=path + 'data/GIS/'
    drv = os.getenv('HOME') + '/DataOthers/'
    pathmeasures = {'Suitability' : pathgis + 'Ramankutty/',
                    'Lights' : drv + '/Lights/tifs/',
                    'Lights2' : drv + '/Lights/tifs/',
                    'Elevation' : drv + '/GLOBE/',
                    'Elevation2' : drv + '/GLOBE/',
                    'RIX' : drv + '/RIX/',
                    'RIX2' : drv + '/RIX/',
                    'CRU' : drv + '/CRU/tif/',
                    'CropsYield' : drv + '/FAO/GAEZ/Agroclimatic/yield/rain_fed/',
                    'CSICrops' : drv + '/FAO/GAEZ/Agroclimatic/energyyield/rain_fed/',
                    'CSI' : drv + '/FAO/GAEZ/Agroclimatic/bestcropcal/CSI/',
                    'CSIWater' : drv + '/FAO/GAEZ/Agroclimatic/bestcropcal/CSI_water/',
                    'CSICycle' : drv + '/FAO/GAEZ/Agroclimatic/bestcropcal/updated/',
                    'CSICycleWater' : drv + '/FAO/GAEZ/Agroclimatic/bestcropcal/updated_water/',
                    'CSIPlow' : drv + '/FAO/GAEZ/Agroclimatic/bestcropcal/plow/',
                    'Malaria' : pathgis + 'Malaria/',
                    'HMI' : drv + '/HMI_Diff/',
                    'PopDensAfrica' : drv + '/UNEP/',
                    'Popdens' : drv + '/GPW/popdens/',
                    'Population' : drv + '/GPW/population/',
                    'HYDE' : drv + '/HYDE/hyde31_final/tif/',
                    'Tsetse' : pathgis + 'Tsetse/',
                    'Ecodiversity' : pathgis + 'Ecological_Zones/',
                    'EcodiversityLGM' : pathgis + 'PaleoVegetation/',
                    'Sea100' : pathcntrycyl,
                    'Coast' : pathcntrycyl,
                    'Inwater' : pathcntrycyl,
                    'PerInwater' : pathcntrycyl,
                    'FluctInwater' : pathcntrycyl,
                    'Landscan' : drv +  '/Landscan/',
                    'GEcon' : os.getenv('HOME')+'/LatexMe/CountryStability/data/GEconRaster/',
                    'HLD' : drv + '/Harmonized Light Data/light/',
}

# Paths to measures available for computations (if adding new measures one needs to change all places where the lists/dicts are created)
mainmeasures = ['Suitability','Lights','Lights2','Elevation','RIX','Elevation2','RIX2','CRU','CSI', 'CSICrops', 'CropsYield',
                'CSICycle','CSIPlow','Malaria','HMI','PopDensAfrica','Popdens','Population',
                'HYDE','Tsetse','Ecodiversity','EcodiversityLGM','Sea100','Coast',
                'Inwater','PerInwater','FluctInwater','Landscan', 'GEcon', 'HLD']

mainmeasuresw = ['Suitability','Lights','Lights2','Elevation','RIX','Elevation2','RIX2','CRU','CSI', 'CSICrops', 'CropsYield', 'CSIWater',
                'CSICycle','CSICycleWater','CSIPlow','Malaria','HMI','PopDensAfrica','Popdens','Population',
                'HYDE','Tsetse','Ecodiversity','EcodiversityLGM','Sea100','Coast',
                'Inwater','PerInwater','FluctInwater','Landscan', 'GEcon', 'HLD']

wgs84measures = ['Lights2','Elevation2','RIX2','PopDensAfrica','Popdens','Population','HYDE','Landscan','CSICrops','CropsYield', 'GEcon', 'HLD']
shpmeasures = ['Ecodiversity','EcodiversityLGM','Sea100','Coast','Inwater','PerInwater','FluctInwater',]
ecodivmeasures = ['Ecodiversity','EcodiversityLGM']
ceameasures = list(set(mainmeasuresw).difference(set(wgs84measures)).difference(set(shpmeasures)))
noecodivmeasures = list(set(ceameasures).difference(set(ecodivmeasures)))

namemeasures = {'Suitability' : -7,
                'Lights' : 7,
                'Lights2' : 7,
                'Elevation' : -4,
                'Elevation2' : -4,
                'RIX' : -4,
                'RIX2' : -4,
                'CRU' : -7,
                'CropsYield' : -4,
                'CSICrops' : -4,
                'CSI' : -7,
                'CSIWater' : -7,
                'CSICycle' : -7,
                'CSICycleWater' : -7,
                'CSIPlow' : -7,
                'Malaria' : -7,
                'HMI' : -4,
                'PopDensAfrica' : -4,
                'Popdens' : -4,
                'Population' : -4,
                'HYDE' : -4,
                'Tsetse' : -7,
                'Ecodiversity' : 0,
                'EcodiversityLGM' : 0,
                'Sea100' : 0,
                'Coast' : 0,
                'Inwater' : 0,
                'PerInwater' : 0,
                'FluctInwater' : 0,
                'Landscan' : -4,
                'GEcon' : -4,
                'HLD' : 0,
                }

# Functions to perform various operations
# Create vector of polygons and iso-codes
def GISData(myshp=pathcntrycyl + 'DCW_countriescyl.shp', adds=False):
    '''
    This function imports the shape file and converts everyhting to Shapely, and WKT and saves it in a
    pandas dataframe
    Additionally, it computes areas and boundary lengths if adds=True
    Usage:
    GISData(shp=file,[adds=adds])
    '''
    # Shape file
    if isinstance(myshp,str):
        data = gp.GeoDataFrame.from_file(myshp)
    elif isinstance(myshp,gp.GeoDataFrame):
        data = myshp.copy()
    return data

def gini(self):
    '''
    gini(geo)

    Return computed Gini coefficient.
    '''
    if self.count()>1:
        xsort = sorted(self.data[self.mask==False].flatten()) # increasing order
        y = np.cumsum(xsort)
        B = sum(y) / (y[-1] * len(xsort))
        return 1 +  1./len(xsort) - 2*B
    else:
        return 1

# The DataFrames to construct and keep all the additional data
mystats=['min', 'max', 'median', 'mean', 'majority', 'sum','std','count']
addstats={'gini':gini}

# Convert project to equal cylindrical area
p=pyproj.Proj(proj='cea',ellps='WGS84')

#wgs84 = {u'datum': u'WGS84', u'no_defs': True, u'proj': u'longlat'}
#cea = {u'datum': u'WGS84', u'lat_ts': 0, u'lon_0': 0, u'no_defs': True, u'proj': u'cea',u'units': u'm',  u'x_0': 0, u'y_0': 0, 'over':True}
wgs84 = CRS("EPSG:4326")
cea = CRS("ESRI:54034")


/tmp/ipykernel_1830624/4139534612.py:30: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gp


In [2]:
df = GISData()

In [3]:
df.crs!=wgs84

True

In [4]:
dfnocyl = df.copy().to_crs(wgs84)

In [5]:
df.crs!=cea

False

In [6]:
df.loc[df.is_valid==False, 'geometry'] = df.loc[df.is_valid==False, 'geometry'].apply(lambda x: x.buffer(0))

In [7]:
dfnocyl.loc[dfnocyl.is_valid==False, 'geometry'] = dfnocyl.loc[dfnocyl.is_valid==False, 'geometry'].apply(lambda x: x.buffer(0))

In [8]:
stats=mystats
copy_properties=True
add_stats=addstats
all_touched=True
adds=True
correct_cea=True
correct_wgs=True

In [9]:
df.head()

,ID,FIPS10_4,COUNTRY,CONTCODE,CONT,F_CODE,F_CODE_DES,OW_ABBREV,ISO_A2,ISO_A3,VMAP_ID,FACTBK_CTY,geometry
0,OC-WS,WS,Samoa,O,Oceania,FA001,Administrative Area,samo,WS,WSM,ADDBDCW_CA000001,Samoa,"MULTIPOLYGON (((-19152958.602 -1518188.249, -1..."
1,OC-FJ,FJ,Fiji,O,Oceania,FA001,Administrative Area,fiji,FJ,FJI,ADDBDCW_CA000002,Fiji,"MULTIPOLYGON (((-20017835.072 -1830741.053, -2..."
2,OC-NZ,NZ,New Zealand,O,Oceania,FA001,Administrative Area,newz,NZ,NZL,ADDBDCW_CA000003,New Zealand,"MULTIPOLYGON (((-19801632.013 -3101423.600, -1..."
3,OC-TO,TN,Tonga,O,Oceania,FA001,Administrative Area,tong,TO,TON,ADDBDCW_CA000004,Tonga,"MULTIPOLYGON (((-19485316.914 -2288511.805, -1..."
4,OC-WF,WF,Wallis and Futuna,O,Oceania,FA001,Administrative Area,wall,WF,WLF,ADDBDCW_CA000005,Wallis and Futuna,"MULTIPOLYGON (((-19606892.604 -1455270.527, -1..."


In [10]:
if adds:
    if df.geom_type.values[0].find('Polygon')!=-1:
        df['area']=df.area/1e6
        df['boundary']=df.boundary.length/1e3
    if df.geom_type.values[0].find('Point')!=-1:
        df['boundary']=df.boundary.length/1e3


In [11]:
df.head()

,ID,FIPS10_4,COUNTRY,CONTCODE,CONT,F_CODE,F_CODE_DES,OW_ABBREV,ISO_A2,ISO_A3,VMAP_ID,FACTBK_CTY,geometry,area,boundary
0,OC-WS,WS,Samoa,O,Oceania,FA001,Administrative Area,samo,WS,WSM,ADDBDCW_CA000001,Samoa,"MULTIPOLYGON (((-19152958.602 -1518188.249, -1...",2875.772157,411.985556
1,OC-FJ,FJ,Fiji,O,Oceania,FA001,Administrative Area,fiji,FJ,FJI,ADDBDCW_CA000002,Fiji,"MULTIPOLYGON (((-20017835.072 -1830741.053, -2...",840.824243,765.802587
2,OC-NZ,NZ,New Zealand,O,Oceania,FA001,Administrative Area,newz,NZ,NZL,ADDBDCW_CA000003,New Zealand,"MULTIPOLYGON (((-19801632.013 -3101423.600, -1...",832.265055,524.499247
3,OC-TO,TN,Tonga,O,Oceania,FA001,Administrative Area,tong,TO,TON,ADDBDCW_CA000004,Tonga,"MULTIPOLYGON (((-19485316.914 -2288511.805, -1...",672.164444,671.848586
4,OC-WF,WF,Wallis and Futuna,O,Oceania,FA001,Administrative Area,wall,WF,WLF,ADDBDCW_CA000005,Wallis and Futuna,"MULTIPOLYGON (((-19606892.604 -1455270.527, -1...",175.027148,111.754664


In [12]:
df['X']=df.centroid.apply(lambda c: c.coords.xy[0][0])
df['Y']=df.centroid.apply(lambda c: c.coords.xy[1][0])
df['lon']=dfnocyl.centroid.apply(lambda c: c.coords.xy[0][0])
df['lat']=dfnocyl.centroid.apply(lambda c: c.coords.xy[1][0])


/tmp/ipykernel_1830624/1197614291.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df['lon']=dfnocyl.centroid.apply(lambda c: c.coords.xy[0][0])
/tmp/ipykernel_1830624/1197614291.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df['lat']=dfnocyl.centroid.apply(lambda c: c.coords.xy[1][0])


# Now how could we speed up other stats?

# Sea

In [13]:
import multiprocessing
from multiprocessing import Pool
from rasterstats import zonal_stats
from affine import Affine

In [14]:
n_procs = os.cpu_count()

## Original code

In [15]:
%%time
measure = 'Sea100'
sea100cyl=GISData(myshp=pathmeasures[measure] + 'coastlcyl_buffer100.shp')

CPU times: user 6.71 s, sys: 172 ms, total: 6.88 s
Wall time: 6.88 s


In [16]:
%%time
sea100cyl['bbox']=sea100cyl.geometry.apply(lambda x: x.bounds)
dfin=df[['geometry','area']].copy()
dfin.crs = df.crs
dfin['bbox']=dfin.geometry.apply(lambda x: x.bounds)
# Create spatial index for sea100cyl
idxsea = index.Index()
[idxsea.insert(sea100cyl.index[i],sea100cyl.bbox[i]) for i in range(sea100cyl.shape[0])]
# Seas with which it intersects
dfin['seas']=dfin.bbox.apply(lambda x:list(idxsea.intersection(x)))
dfin['sea']=dfin.seas.apply(lambda x: unary_union(sea100cyl.geometry.iloc[x]))
# Functions to extract area and length
def seaintersects(row):
    if row['sea']!=[]:
        area=row['geometry'].intersection(row['sea']).area/1e6
    else:
        area=0
    return area
dfin['sea100']=dfin.apply(lambda row: seaintersects(row),axis=1)
dfin['sea100pct']=dfin['sea100']/dfin['area']
#df=pd.merge(df,dfin[['sea100','sea100pct']],left_index=True,right_index=True, how='outer')

CPU times: user 15.9 s, sys: 398 ms, total: 16.3 s
Wall time: 16 s


In [17]:
dfin_or = dfin.copy()

## Parallelizing

In [18]:
# Functions to extract area and length
def seaintersects(row):
    if row['sea']!=[]:
        area=row['geometry'].intersection(row['sea']).area/1e6
    else:
        area=0
    return area

def compute_seas(sea100cyl):
    sea100cyl['bbox'] = sea100cyl.geometry.apply(lambda x: x.bounds)
    idxsea = index.Index()
    [idxsea.insert(i, sea100cyl.bbox[i]) for i in sea100cyl.index]
    return idxsea

def process_geometry(dfin):
    dfin['bbox'] = dfin.geometry.apply(lambda x: x.bounds)
    dfin['seas'] = dfin.bbox.apply(lambda x: list(idxsea.intersection(x)))
    dfin['sea'] = dfin.seas.apply(lambda x: unary_union(sea100cyl.geometry.iloc[x]))
    dfin['sea100']=dfin.apply(lambda row: seaintersects(row),axis=1)
    dfin['sea100pct']=dfin['sea100']/dfin['area']    
    return dfin

In [19]:
%%time
idxsea = compute_seas(sea100cyl)

CPU times: user 3.12 s, sys: 174 ms, total: 3.29 s
Wall time: 3.1 s


In [21]:
%%time
with Pool(processes=n_procs, initializer=compute_seas, initargs=(sea100cyl,)) as pool:
    # Parallelize the geometry processing
    dfin_chunks = np.array_split(dfin, n_procs)
    processed_dfin = pool.map(process_geometry, [chunk for chunk in dfin_chunks])

# Combine the results if needed
combined_dfin = pd.concat(processed_dfin, ignore_index=True)

CPU times: user 863 ms, sys: 1.49 s, total: 2.36 s
Wall time: 8.52 s


In [22]:
combined_dfin.head()

,geometry,area,bbox,seas,sea,sea100,sea100pct
0,"MULTIPOLYGON (((-19152958.602 -1518188.249, -1...",2875.772157,"(-19235851.99270739, -1537477.4377452706, -190...","[29222, 29224, 29218, 29216, 29215, 29217, 29214]",POLYGON ((-18998594.466086853 -1458567.3119530...,2875.772157,1.0
1,"MULTIPOLYGON (((-20017835.072 -1830741.053, -2...",840.824243,"(-20037508.231469758, -2238178.1814031964, -19...","[29235, 29234, 29233, 29231, 29227, 29267, 292...",POLYGON ((-20048048.276682395 -2161373.3609936...,840.824243,1.0
2,"MULTIPOLYGON (((-19801632.013 -3101423.600, -1...",832.265055,"(-19884795.576263785, -4445632.550982113, -195...","[2060, 2059, 2058, 2055, 2057, 2054, 2053, 205...",MULTIPOLYGON (((-19688159.23802806 -4518386.16...,832.265055,1.0
3,"MULTIPOLYGON (((-19485316.914 -2288511.805, -1...",672.164444,"(-19615900.242943525, -2410252.874714202, -193...","[29355, 29349, 29363, 29368, 29369, 29354, 293...",MULTIPOLYGON (((-19710902.30286037 -2378442.65...,672.164444,1.0
4,"MULTIPOLYGON (((-19606892.604 -1455270.527, -1...",175.027148,"(-19837888.673425876, -1574685.9202680017, -19...","[29225, 29223, 29209]",MULTIPOLYGON (((-19774462.418312743 -1478407.3...,175.027148,1.0


In [23]:
dfin_or.head()

,geometry,area,bbox,seas,sea,sea100,sea100pct
0,"MULTIPOLYGON (((-19152958.602 -1518188.249, -1...",2875.772157,"(-19235851.99270739, -1537477.4377452706, -190...","[29222, 29224, 29218, 29216, 29215, 29217, 29214]",POLYGON ((-18998594.466086853 -1458567.3119530...,2875.772157,1.0
1,"MULTIPOLYGON (((-20017835.072 -1830741.053, -2...",840.824243,"(-20037508.231469758, -2238178.1814031964, -19...","[29235, 29234, 29233, 29231, 29227, 29267, 292...",POLYGON ((-20048048.276682395 -2161373.3609936...,840.824243,1.0
2,"MULTIPOLYGON (((-19801632.013 -3101423.600, -1...",832.265055,"(-19884795.576263785, -4445632.550982113, -195...","[2060, 2059, 2058, 2055, 2057, 2054, 2053, 205...",MULTIPOLYGON (((-19688159.23802806 -4518386.16...,832.265055,1.0
3,"MULTIPOLYGON (((-19485316.914 -2288511.805, -1...",672.164444,"(-19615900.242943525, -2410252.874714202, -193...","[29355, 29349, 29363, 29368, 29369, 29354, 293...",MULTIPOLYGON (((-19710902.30286037 -2378442.65...,672.164444,1.0
4,"MULTIPOLYGON (((-19606892.604 -1455270.527, -1...",175.027148,"(-19837888.673425876, -1574685.9202680017, -19...","[29225, 29223, 29209]",MULTIPOLYGON (((-19774462.418312743 -1478407.3...,175.027148,1.0


In [24]:
pd.concat([combined_dfin[['sea100', 'sea100pct']], dfin_or[['sea100', 'sea100pct']]], axis=1).corr()

,sea100,sea100pct,sea100,sea100pct
sea100,1.000000,-0.142251,1.000000,-0.142251
sea100pct,-0.142251,1.000000,-0.142251,1.000000
sea100,1.000000,-0.142251,1.000000,-0.142251
sea100pct,-0.142251,1.000000,-0.142251,1.000000


## Both generate the same output, but parallelized is 50% faster

# Added code to geostats sharing dfin, idxsea, and sea100cyl in args
## MUCH SLOWER and GENERATES WRONG OUTPUT!

In [25]:
import geostats

context has already been set


In [26]:
myseas = geostats.geostats(geostats.GISData(), measures=['Sea100'])

/users/ozak/Python/shared/geostats/main.py:400: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  self.df['lon']=self.dfnocyl.centroid.apply(lambda c: c.coords.xy[0][0])
/users/ozak/Python/shared/geostats/main.py:401: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  self.df['lat']=self.dfnocyl.centroid.apply(lambda c: c.coords.xy[1][0])


In [27]:
%%time
myseas.geostats()

Computing Stats for: Sea100
CPU times: user 37.7 s, sys: 2.4 s, total: 40.1 s
Wall time: 44.5 s


In [28]:
myseas.df

,ID,FIPS10_4,COUNTRY,CONTCODE,CONT,F_CODE,F_CODE_DES,OW_ABBREV,ISO_A2,ISO_A3,...,FACTBK_CTY,geometry,area,boundary,X,Y,lon,lat,sea100,sea100pct
0,OC-WS,WS,Samoa,O,Oceania,FA001,Administrative Area,samo,WS,WSM,...,Samoa,"MULTIPOLYGON (((-19152958.602 -1518188.249, -1...",2875.772157,411.985556,-1.916567e+07,-1.504961e+06,-172.167871,-13.738431,0.0,0.0
1,OC-FJ,FJ,Fiji,O,Oceania,FA001,Administrative Area,fiji,FJ,FJI,...,Fiji,"MULTIPOLYGON (((-20017835.072 -1830741.053, -2...",840.824243,765.802587,-1.997896e+07,-1.893087e+06,-179.472083,-17.385886,0.0,0.0
2,OC-NZ,NZ,New Zealand,O,Oceania,FA001,Administrative Area,newz,NZ,NZL,...,New Zealand,"MULTIPOLYGON (((-19801632.013 -3101423.600, -1...",832.265055,524.499247,-1.965771e+07,-4.351866e+06,-176.577984,-43.435529,0.0,0.0
3,OC-TO,TN,Tonga,O,Oceania,FA001,Administrative Area,tong,TO,TON,...,Tonga,"MULTIPOLYGON (((-19485316.914 -2288511.805, -1...",672.164444,671.848586,-1.946370e+07,-2.150382e+06,-174.847165,-19.858982,0.0,0.0
4,OC-WF,WF,Wallis and Futuna,O,Oceania,FA001,Administrative Area,wall,WF,WLF,...,Wallis and Futuna,"MULTIPOLYGON (((-19606892.604 -1455270.527, -1...",175.027148,111.754664,-1.972495e+07,-1.514557e+06,-177.194346,-13.829204,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,NA-SX,NN,Sint Maarten,N,North America,FA001,Administrative Area,nets,SX,SXM,...,Sint Maarten,"POLYGON ((-7015125.034 1964868.791, -7015280.8...",41.389139,37.842096,-7.020605e+06,1.963228e+06,-63.067167,18.044022,0.0,0.0
266,SA-CW,UC,Curaçao,N,North America,FA001,Administrative Area,nets,CW,CUW,...,Curacao,"MULTIPOLYGON (((-7641138.762 1314625.315, -764...",447.246703,145.137458,-7.677296e+06,1.337395e+06,-68.966363,12.184257,0.0,0.0
267,NA-GP,GP,Guadeloupe,N,North America,FA001,Administrative Area,guad,GP,GLP,...,Guadeloupe,"MULTIPOLYGON (((-6855038.700 1729691.315, -685...",1673.333038,397.514877,-6.850164e+06,1.768030e+06,-61.536063,16.199080,0.0,0.0
268,NA-MF,RN,Saint Martin,N,North America,FA001,Administrative Area,saim,MF,MAF,...,Saint Martin,"MULTIPOLYGON (((-7017500.926 1972314.099, -701...",54.702574,51.442846,-7.020320e+06,1.967477e+06,-63.064609,18.084388,0.0,0.0


## Another possibility (but is much slower and does not seem to run inside geostats object)

In [30]:
class ParallelProcessor:
    def __init__(self, sea100cyl, df, num_processes=n_procs):
        self.num_processes = num_processes
        self.sea100cyl = sea100cyl
        self.df = df
        self._compute_seas()

    def _compute_seas(self):
        self.sea100cyl['bbox'] = self.sea100cyl.geometry.apply(lambda x: x.bounds)
        self.idxsea = index.Index()
        [self.idxsea.insert(i, self.sea100cyl.bbox[i]) for i in self.sea100cyl.index]

    def _process_geometry(self, dfin_chunk):
        dfin_chunk['bbox'] = dfin_chunk.geometry.apply(lambda x: x.bounds)
        dfin_chunk['seas'] = dfin_chunk.bbox.apply(lambda x: list(self.idxsea.intersection(x)))
        dfin_chunk['sea'] = dfin_chunk.seas.apply(lambda x: unary_union(self.sea100cyl.geometry.iloc[x]))
        return dfin_chunk

    def process(self):
        with Pool(processes=self.num_processes) as pool:
            dfin_chunks = np.array_split(self.df, self.num_processes)
            processed_dfin = pool.map(self._process_geometry, dfin_chunks)

        combined_dfin = pd.concat(processed_dfin, ignore_index=True)
        return combined_dfin

In [31]:
%%time
# Assuming you have sea100cyl and self.df defined
parallel_processor = ParallelProcessor(sea100cyl, dfin)
result = parallel_processor.process()

CPU times: user 50.5 s, sys: 8.19 s, total: 58.7 s
Wall time: 1min 14s
